<a href="https://colab.research.google.com/github/napoles-uach/streamlit_apps/blob/main/Streamlit_Colab/09_Streamlit__Colab_Networkx_Pyvis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial 9. Make interactive graphs in your streamlit app. 
## This notebook shows how to deploy an Streamlit app on top of Google Colaboratory + Networkx + Pyvis.

![](https://github.com/napoles-uach/streamlit_apps/blob/main/Streamlit_Colab/Images/09.png?raw=True)

## 1) Run all and click the "**Link to web app**" (ignore the output of the last cell).
## 2) Modify the code in `app.py`


# Setup
(Do not need to install networkx.) 

In [1]:
#@title -----------> Installation of Streamlit and pyngrok of course!!
!pip -q install streamlit
!pip -q install pyngrok

     |████████████████████████████████| 7.4MB 5.0MB/s 
     |████████████████████████████████| 163kB 46.3MB/s 
     |████████████████████████████████| 112kB 49.5MB/s 
     |████████████████████████████████| 4.5MB 48.0MB/s 
     |████████████████████████████████| 81kB 8.9MB/s 
     |████████████████████████████████| 71kB 6.9MB/s 
     |████████████████████████████████| 122kB 41.2MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.4.2 which is incompatible.


In [2]:
!pip install pyvis

Here we use the ngrok cloud service. Don't change this!!

The following is the `app.py` base code. It can be modified in the cell or in the folder to the left.

In [3]:
%%writefile app.py
import streamlit as st
import streamlit.components.v1 as components
import networkx as nx
import matplotlib.pyplot as plt
from pyvis.network import Network
import got 
#Network(notebook=True)
st.title('Hello Pyvis')
# make Network show itself with repr_html

#def net_repr_html(self):
#  nodes, edges, height, width, options = self.get_network_data()
#  html = self.template.render(height=height, width=width, nodes=nodes, edges=edges, options=options)
#  return html

#Network._repr_html_ = net_repr_html
st.sidebar.title('Choose your favorite Graph')
option=st.sidebar.selectbox('select graph',('Simple','Karate', 'GOT','Random'))
physics=st.sidebar.checkbox('add physics interactivity?')
got.simple_func(physics)

if option=='Simple':
  HtmlFile = open("test.html", 'r', encoding='utf-8')
  source_code = HtmlFile.read() 
  components.html(source_code, height = 900,width=900)


got.got_func(physics)

if option=='GOT':
  HtmlFile = open("gameofthrones.html", 'r', encoding='utf-8')
  source_code = HtmlFile.read() 
  components.html(source_code, height = 1200,width=1000)



got.karate_func(physics)

if option=='Karate':
  HtmlFile = open("karate.html", 'r', encoding='utf-8')
  source_code = HtmlFile.read() 
  components.html(source_code, height = 1200,width=1000)


got.random_func(physics)

if option=='Random':
  HtmlFile = open("random.html", 'r', encoding='utf-8')
  source_code = HtmlFile.read() 
  components.html(source_code, height = 1200,width=1000)
# Put your Python+Streamlit code here ...
# you can modify it by double cliking on the folder icon at the left

Writing app.py


This last cell would keep the app running. If stoped, the app woul be disconnected.

In [4]:
%%writefile got.py
import networkx as nx
import matplotlib.pyplot as plt
from pyvis.network import Network
import pandas as pd
import streamlit as st


def got_func(physics):
  got_net = Network(height="600px", width="100%", font_color="black",heading='Game of Thrones Graph')

# set the physics layout of the network
  got_net.barnes_hut()
  got_data = pd.read_csv("https://www.macalester.edu/~abeverid/data/stormofswords.csv")
  #got_data = pd.read_csv("stormofswords.csv")
  #got_data.rename(index={0: "Source", 1: "Target", 2: "Weight"}) 
  sources = got_data['Source']
  targets = got_data['Target']
  weights = got_data['Weight']

  edge_data = zip(sources, targets, weights)

  for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]

    got_net.add_node(src, src, title=src)
    got_net.add_node(dst, dst, title=dst)
    got_net.add_edge(src, dst, value=w)

  neighbor_map = got_net.get_adj_list()

# add neighbor data to node hover data
  for node in got_net.nodes:
    node["title"] += " Neighbors:<br>" + "<br>".join(neighbor_map[node["id"]])
    node["value"] = len(neighbor_map[node["id"]])
  if physics:
    got_net.show_buttons(filter_=['physics'])
  got_net.show("gameofthrones.html")
  

def simple_func(physics): 
  nx_graph = nx.cycle_graph(10)
  nx_graph.nodes[1]['title'] = 'Number 1'
  nx_graph.nodes[1]['group'] = 1
  nx_graph.nodes[3]['title'] = 'I belong to a different group!'
  nx_graph.nodes[3]['group'] = 10
  nx_graph.add_node(20, size=20, title='couple', group=2)
  nx_graph.add_node(21, size=15, title='couple', group=2)
  nx_graph.add_edge(20, 21, weight=5)
  nx_graph.add_node(25, size=25, label='lonely', title='lonely node', group=3)


  nt = Network("500px", "500px",notebook=True,heading='')
  nt.from_nx(nx_graph)
  #physics=st.sidebar.checkbox('add physics interactivity?')
  if physics:
    nt.show_buttons(filter_=['physics'])
  nt.show('test.html')


def karate_func(physics): 
  G = nx.karate_club_graph()


  nt = Network("500px", "500px",notebook=True,heading='Zachary’s Karate Club graph')
  nt.from_nx(G)
  #physics=st.sidebar.checkbox('add physics interactivity?')
  if physics:
    nt.show_buttons(filter_=['physics'])
  nt.show('karate.html')


def random_func(physics): 
  G = nx.random_tree(20)


  nt = Network("500px", "500px",directed=True,heading='Random graph')
  nt.from_nx(G)
  #physics=st.sidebar.checkbox('add physics interactivity?')
  if physics:
    nt.show_buttons(filter_=['physics'])
  nt.show('random.html')

Writing got.py


In [5]:
#@title This last cell would keep the app running. If stoped, the app would be disconnected.
from pyngrok import ngrok
public_url = ngrok.connect(port='80')
print('Link to web app:')
print (public_url)
!streamlit run --server.port 80 app.py >/dev/null

Link to web app:
NgrokTunnel: "http://8318e1131c89.ngrok.io" -> "http://localhost:80"
